##Credit Card Fraud Detection

d
**Data**: The dataset covers credit card transactions done by European cardholders in September 2013.
In this dataset, we have 492 frauds out of 284,807 transactions that happened in two days. The dataset is heavily skewed, with the positive class (frauds) accounting for just 0.172 percent of all transactions. It contains only PCA transformation result as numerical input variables. The major components derived with PCA are features V1, V2,... V28; the only features not changed with PCA are 'Time' and 'Amount.' The seconds elapsed between each transaction and the first transaction in the dataset are stored in the 'Time' field. The transaction Amount is represented by the feature 'Amount'. The feature 'Class' has a value of 1 when there is a fraud and 0 when there isn't. This dataset is from [Kaggle- credit card fraud detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).
             
**Goal**: The goal of this approach is to maintain or improve the performance of classifiers in the context of excessively imbalanced and large datasets using Big Data techniques.

**Approach**: This global approach first clusters the data using k-means clustering, then performs SMOTE + ENN on these clusters, before they are merged together using union and evaluated using a Random Forest classifier.

###Import required packages

In [0]:
# Import all the required libraries
from pyspark.sql import functions as F
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors
import random
import numpy as np
from functools import reduce
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.functions import rand,col,when,concat,substring,lit,udf,lower,sum as ps_sum,count as ps_count,row_number
from pyspark.sql.window import *
from pyspark.sql import DataFrame
from pyspark.ml.feature import VectorAssembler,BucketedRandomProjectionLSH,VectorSlicer
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors,VectorUDT
from pyspark.sql.functions import array, create_map, struct
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
import random
import numpy as np
from pyspark.sql import Row
from sklearn import neighbors
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator


###Load and understand the dataset

In [0]:
# Load the dataset which is in Comma-Separated Value (CSV) format.
df = spark.read.option("header",True).csv("/mnt/sjjhkkdf/creditcard.csv")

# Cache DataFrame so that we only read it from disk once
df.cache()

#Check the name, datatype and values of columns
df.take(1)

Out[79]: [Row(Time='0', V1='-1.3598071336738', V2='-0.0727811733098497', V3='2.53634673796914', V4='1.37815522427443', V5='-0.338320769942518', V6='0.462387777762292', V7='0.239598554061257', V8='0.0986979012610507', V9='0.363786969611213', V10='0.0907941719789316', V11='-0.551599533260813', V12='-0.617800855762348', V13='-0.991389847235408', V14='-0.311169353699879', V15='1.46817697209427', V16='-0.470400525259478', V17='0.207971241929242', V18='0.0257905801985591', V19='0.403992960255733', V20='0.251412098239705', V21='-0.018306777944153', V22='0.277837575558899', V23='-0.110473910188767', V24='0.0669280749146731', V25='0.128539358273528', V26='-0.189114843888824', V27='0.133558376740387', V28='-0.0210530534538215', Amount='149.62', Class='0')]

####Data description
**Attribute information**:
```
The features V1, V2,... V28 are derived from PCA transformation.

'Time'- The seconds elapsed between each transaction and the first transaction in the dataset 

'Amount'- The transaction Amount 

'Class'- a value of 1 when there is a fraud and 0 when there isn't.

```
**The target variable is the class of the transaction i.e. fraud (1) or not (0).**

##Data preparation

As all the features are in string, cast all features to floats except Time and Class

####Casting

In [0]:
#get the list of name of the columns except 'Time' and 'Class'
targets = [i for i in df.schema.names if i not in ["Time", "Class"]]
for col in targets: 
  df = df.withColumn(col, df['`{}`'.format(col)].cast('float'))

#Cast 'Time' and 'Class' to integer
for col in ["Time", "Class"]:
  df = df.withColumn(col, df['`{}`'.format(col)].cast('integer'))

#Check the datatype of columns
df.printSchema()

#Visualize the data
# display(df)

root
-- Time: integer (nullable = true)
-- V1: float (nullable = true)
-- V2: float (nullable = true)
-- V3: float (nullable = true)
-- V4: float (nullable = true)
-- V5: float (nullable = true)
-- V6: float (nullable = true)
-- V7: float (nullable = true)
-- V8: float (nullable = true)
-- V9: float (nullable = true)
-- V10: float (nullable = true)
-- V11: float (nullable = true)
-- V12: float (nullable = true)
-- V13: float (nullable = true)
-- V14: float (nullable = true)
-- V15: float (nullable = true)
-- V16: float (nullable = true)
-- V17: float (nullable = true)
-- V18: float (nullable = true)
-- V19: float (nullable = true)
-- V20: float (nullable = true)
-- V21: float (nullable = true)
-- V22: float (nullable = true)
-- V23: float (nullable = true)
-- V24: float (nullable = true)
-- V25: float (nullable = true)
-- V26: float (nullable = true)
-- V27: float (nullable = true)
-- V28: float (nullable = true)
-- Amount: float (nullable = true)
-- Class: integer (nullable = true)

#### Check whether the dataset is balanced or not
Total fraud transactions in the dataset

In [0]:
df.agg(F.sum("Class")).collect()[0][0]

Out[81]: 492

### Split data into training and test sets

Split the dataframe df into 70% for training and 30% for test.

In [0]:

df2=df
df2_train, df2_test = df2.randomSplit([0.7,0.3], seed=1)
feature_cols = df2.columns
feature_cols.remove("Class") # Remove output feature 

assembler = VectorAssembler(
  inputCols= ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount"], 
  outputCol= "features"
)

output = assembler.transform(df2_train)
print("Assembled predictor columns to vector column 'features'")
output.select("features", "Class").show(truncate=True)



Assembled predictor columns to vector column 'features'
+--------------------+-----+
 features|Class|
+--------------------+-----+
[-1.3598071336746...| 0|
[1.19185709953308...| 0|
[-1.3583540916442...| 0|
[-0.9662716984748...| 0|
[1.22965764999389...| 0|
[-0.8942860960960...| 0|
[-0.6442694664001...| 0|
[-0.3382617533206...| 0|
[0.38497820496559...| 0|
[1.24999868869781...| 0|
[1.44904375076293...| 0|
[-5.4012575149536...| 0|
[1.49293601512908...| 0|
[0.69488477706909...| 0|
[0.96249604225158...| 0|
[0.24749112129211...| 0|
[1.16661643981933...| 0|
[-2.0742945671081...| 0|
[-1.9465250968933...| 0|
[-0.4142888188362...| 0|
+--------------------+-----+
only showing top 20 rows

# Functions

Functions used:
  - Pre-SMOTE
  - SMOTE
  - ENN

## SMOTE (Synthetic Minority Over-sampling Technique)  and pre-SMOTE function



SMOTE function uses LSH with Bucketed Random Projection. The SMOTE used is a global approach. 

The code was taken from:https://gist.github.com/hwang018/420e288021e9bdacd133076600a9ea8c

###Pre-SMOTE
- pre-smote function - checks for features which are categorical and numerical, then pre process accordingly then vectorise the features.

In [0]:
# Taken from: https://gist.github.com/hwang018/420e288021e9bdacd133076600a9ea8c
#for categorical columns, must take its stringIndexed form (smote should be after string indexing, default by frequency)

def pre_smote_df_process(df,num_cols,cat_cols,target_col,index_suffix="_index"):
    '''
    string indexer and vector assembler
    inputs:
    * df: spark df, original
    * num_cols: numerical cols to be assembled
    * cat_cols: categorical cols to be stringindexed
    * target_col: prediction target
    * index_suffix: will be the suffix after string indexing
    output:
    * vectorized: spark df, after stringindex and vector assemble, ready for smote
    '''
    if(df.select(target_col).distinct().count() != 2):
        raise ValueError("Target col must have exactly 2 classes")
        
    if target_col in num_cols:
        num_cols.remove(target_col)

    # only assembled numeric columns into features
    assembler = VectorAssembler(inputCols = num_cols, outputCol = 'features')
    # index the string cols, except possibly for the label col
    assemble_stages = [StringIndexer(inputCol=column, outputCol=column+index_suffix).fit(df) for column in list(set(cat_cols)-set([target_col]))]
    # add the stage of numerical vector assembler
    assemble_stages.append(assembler.setHandleInvalid("skip"))
    pipeline = Pipeline(stages=assemble_stages)
    pos_vectorized = pipeline.fit(df).transform(df)
    
    # drop original num cols and cat cols
    drop_cols = num_cols+cat_cols
    
    keep_cols = [a for a in pos_vectorized.columns if a not in drop_cols]
    
    vectorized = pos_vectorized.select(*keep_cols).withColumn('label',pos_vectorized[target_col]).drop(target_col)
    
    return vectorized

### SMOTE (Synthetic Minority Over-sampling Technique) 

- SMOTE function uses LSH with Bucketed Random Projection. The SMOTE used is a global approach.

In [0]:
# From https://gist.github.com/hwang018/420e288021e9bdacd133076600a9ea8c



  
 ############################## spark smote oversampling ##########################
def smote(vectorized_sdf,smote_config):
    '''
    contains logic to perform smote oversampling, given a spark df with 2 classes
    inputs:
    * vectorized_sdf: cat cols are already stringindexed, num cols are assembled into 'features' vector
      df target col should be 'label'
    * smote_config: config obj containing smote parameters
    output:
    * oversampled_df: spark df after smote oversampling
    '''
    dataInput_min = vectorized_sdf[vectorized_sdf['label'] == 1]
    dataInput_maj = vectorized_sdf[vectorized_sdf['label'] == 0]
    
    # LSH, bucketed random projection
    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes",seed=smote_config.seed, bucketLength=smote_config.bucketLength)
    # smote only applies on existing minority instances    
    model = brp.fit(dataInput_min)
    model.transform(dataInput_min)

    # here distance is calculated from brp's param inputCol
    self_join_w_distance = model.approxSimilarityJoin(dataInput_min, dataInput_min, float("inf"), distCol="EuclideanDistance")

    # remove self-comparison (distance 0)
    self_join_w_distance = self_join_w_distance.filter(self_join_w_distance.EuclideanDistance > 0)

    over_original_rows = Window.partitionBy("datasetA").orderBy("EuclideanDistance")

    self_similarity_df = self_join_w_distance.withColumn("r_num", F.row_number().over(over_original_rows))

    self_similarity_df_selected = self_similarity_df.filter(self_similarity_df.r_num <= smote_config.k)

    over_original_rows_no_order = Window.partitionBy('datasetA')

    # list to store batches of synthetic data
    res = []
    
    # two udf for vector add and subtract, subtraction include a random factor [0,1]
    subtract_vector_udf = F.udf(lambda arr: random.uniform(0, 1)*(arr[0]-arr[1]), VectorUDT())
    add_vector_udf = F.udf(lambda arr: arr[0]+arr[1], VectorUDT())
    
    # retain original columns
    original_cols = dataInput_min.columns
    
    for i in range(smote_config.multiplier):
        print("generating batch %s of synthetic instances"%i)
        # logic to randomly select neighbour: pick the largest random number generated row as the neighbour
        df_random_sel = self_similarity_df_selected.withColumn("rand", F.rand()).withColumn('max_rand', F.max('rand').over(over_original_rows_no_order))\
                            .where(F.col('rand') == F.col('max_rand')).drop(*['max_rand','rand','r_num'])
        # create synthetic feature numerical part
        df_vec_diff = df_random_sel.select('*', subtract_vector_udf(F.array('datasetA.features', 'datasetB.features')).alias('vec_diff'))
        df_vec_modified = df_vec_diff.select('*', add_vector_udf(F.array('datasetA.features', 'vec_diff')).alias('features'))
        
        # for categorical cols, either pick original or the neighbour's cat values
        for c in original_cols:
            # randomly select neighbour or original data
            col_sub = random.choice(['datasetA','datasetB'])
            val = "{0}.{1}".format(col_sub,c)
            if c != 'features':
                # do not unpack original numerical features
                df_vec_modified = df_vec_modified.withColumn(c,F.col(val))
        
        # this df_vec_modified is the synthetic minority instances,
        df_vec_modified = df_vec_modified.drop(*['datasetA','datasetB','vec_diff','EuclideanDistance'])
        
        res.append(df_vec_modified)
    
    dfunion = reduce(DataFrame.unionAll, res)
    # union synthetic instances with original full (both minority and majority) df
    oversampled_df = dfunion.union(vectorized_sdf.select(dfunion.columns))
    
    return oversampled_df


## ENN
- ENN function uses LSH with Bucketed Random Projection. The ENN used is a global approach. This ENN code is inspired from SMOTE code above

In [0]:
#ENN
df_enn=[]
def enn(vectorized_sdf,smote_config):
    dataInput_min = vectorized_sdf[vectorized_sdf['label'] == 1]
    dataInput_maj = vectorized_sdf[vectorized_sdf['label'] == 0]
    
    # LSH, bucketed random projection
    brp = BucketedRandomProjectionLSH(inputCol="features",outputCol="hashes",seed=smote_config.seed,bucketLength=smote_config.bucketLength)
    # smote only applies on existing minority instances    
    model = brp.fit(vectorized_sdf)
    model.transform(vectorized_sdf)

    # here distance is calculated from brp's param inputCol
    self_join_w_distance = model.approxSimilarityJoin(vectorized_sdf, vectorized_sdf, float("inf"), distCol="EuclideanDistance")

    # remove self-comparison (distance 0)
    self_join_w_distance = self_join_w_distance.filter(self_join_w_distance.EuclideanDistance > 0)

    over_original_rows = Window.partitionBy("datasetA").orderBy("EuclideanDistance")

    self_similarity_df = self_join_w_distance.withColumn("r_num", F.row_number().over(over_original_rows))
    
    # finding the k nearest neighbours
    self_similarity_df_selected = self_similarity_df.filter(self_similarity_df.r_num <= smote_config.k)
    
    self_similarity_df_selected1=self_similarity_df_selected.withColumn("label",F.col("datasetA.label"))
    
    self_similarity_df_selected1=self_similarity_df_selected1.withColumn("labelB",F.col("datasetB.label"))
    
 # checking if the class is same   
    self_similarity_df_selected1=self_similarity_df_selected1.withColumn('flag', F.when((F.col("label") == F.col("labelB")), 0).otherwise(1))
            
    windowSpec = Window.partitionBy().orderBy('flag')
    
    self_similarity_df_selected1 = self_similarity_df_selected1.withColumn("r_num2", F.row_number().over(windowSpec))
    

    self_similarity_df_selected1
                             
    win = Window.partitionBy("datasetA").orderBy("r_num2")                               
    prev = F.lag(F.col("Flag")).over(win)
    prev_2 = F.lag(F.col("Flag"), 2).over(win) 
    a = self_similarity_df_selected1.withColumn("Flag_1", prev)
    a = a.withColumn("Flag_2", prev_2)
    
     # removing if the class is not the same
    a=a.filter(F.col("r_num") ==3)
    a=a.filter((F.col("Flag_1")+F.col("Flag_2")+F.col("flag"))==0)
    
    a=a.withColumn("features",F.col("datasetA.features"))
    a=a.drop(*['datasetA','datasetB','r_num','EuclideanDistance','r_num2','labelB','flag','Flag_2','Flag_1'])
    return a


# Pre-Processing

### Vectorising the features using the pre smote function

In [0]:

a = list(df2_train.columns)

df2_train = pre_smote_df_process(df2_train, a, cat_cols=[], target_col = "Class" )
df2_train.cache()

df2_test = pre_smote_df_process(df2_test, a, cat_cols=[], target_col = "Class" )
df2_test.cache()


display(df2_train)


features,label
"Map(vectorType -> dense, length -> 30, values -> List(0.0, -1.3598071336746216, -0.07278117537498474, 2.536346673965454, 1.37815523147583, -0.3383207619190216, 0.4623877704143524, 0.23959855735301971, 0.09869790077209473, 0.3637869656085968, 0.0907941684126854, -0.5515995621681213, -0.6178008317947388, -0.9913898706436157, -0.3111693561077118, 1.4681769609451294, -0.47040051221847534, 0.2079712450504303, 0.025790579617023468, 0.4039929509162903, 0.2514120936393738, -0.018306778743863106, 0.2778375744819641, -0.11047390848398209, 0.06692807376384735, 0.12853935360908508, -0.18911483883857727, 0.13355837762355804, -0.021053053438663483, 149.6199951171875))",0
"Map(vectorType -> dense, length -> 30, values -> List(0.0, 1.191857099533081, 0.26615071296691895, 0.16648010909557343, 0.448154091835022, 0.060017649084329605, -0.08236081153154373, -0.07880298048257828, 0.0851016566157341, -0.2554251253604889, -0.16697441041469574, 1.6127266883850098, 1.0652352571487427, 0.489095002412796, -0.14377228915691376, 0.6355580687522888, 0.4639170467853546, -0.11480466276407242, -0.18336127698421478, -0.14578303694725037, -0.06908313184976578, -0.22577524185180664, -0.6386719346046448, 0.10128802061080933, -0.3398464620113373, 0.16717040538787842, 0.12589453160762787, -0.008983098901808262, 0.014724168926477432, 2.690000057220459))",0
"Map(vectorType -> dense, length -> 30, values -> List(1.0, -1.358354091644287, -1.3401631116867065, 1.7732093334197998, 0.3797796070575714, -0.5031981468200684, 1.800499439239502, 0.7914609313011169, 0.2476757913827896, -1.514654278755188, 0.20764286816120148, 0.6245014667510986, 0.06608368456363678, 0.7172927260398865, -0.16594591736793518, 2.34586501121521, -2.8900833129882812, 1.1099693775177002, -0.12135931104421616, -2.261857032775879, 0.5249797105789185, 0.24799814820289612, 0.7716794013977051, 0.9094122648239136, -0.6892809271812439, -0.3276418447494507, -0.13909657299518585, -0.055352795869112015, -0.05975184217095375, 378.6600036621094))",0
"Map(vectorType -> dense, length -> 30, values -> List(1.0, -0.966271698474884, -0.18522600829601288, 1.7929933071136475, -0.8632912635803223, -0.010308879427611828, 1.2472031116485596, 0.23760893940925598, 0.3774358630180359, -1.3870240449905396, -0.05495192110538483, -0.22648726403713226, 0.1782282292842865, 0.5077568888664246, -0.28792375326156616, -0.6314181089401245, -1.0596472024917603, -0.6840927600860596, 1.9657750129699707, -1.2326220273971558, -0.20803777873516083, -0.10830045491456985, 0.005273596849292517, -0.19032052159309387, -1.1755753755569458, 0.6473760604858398, -0.22192884981632233, 0.06272284686565399, 0.06145763024687767, 123.5))",0
"Map(vectorType -> dense, length -> 30, values -> List(4.0, 1.2296576499938965, 0.14100350439548492, 0.04537077248096466, 1.2026127576828003, 0.1918809860944748, 0.27270811796188354, -0.0051590027287602425, 0.08121293783187866, 0.4649600088596344, -0.0992543175816536, -1.4169071912765503, -0.1538258194923401, -0.7510626912117004, 0.1673719584941864, 0.05014359578490257, -0.4435867965221405, 0.0028205125126987696, -0.6119873523712158, -0.04557504504919052, -0.21963255107402802, -0.16771626472473145, -0.27070972323417664, -0.15410378575325012, -0.7800554037094116, 0.7501369118690491, -0.2572368383407593, 0.03450743108987808, 0.005167768802493811, 4.989999771118164))",0
"Map(vectorType -> dense, length -> 30, values -> List(7.0, -0.8942860960960388, 0.28615719079971313, -0.11319221556186676, -0.27152612805366516, 2.6695985794067383, 3.721817970275879, 0.37014514207839966, 0.8510844707489014, -0.39204758405685425, -0.4104304313659668, -0.7051165699958801, -0.11045226454734802, -0.2862536311149597, 0.0743553638458252, -0.3287830650806427, -0.21007727086544037, -0.4997679591178894, 0.11876486241817474, 0.5703281760215759, 0.052735667675733566, -0.07342509925365448, -0.26809161901474, -0.20423266291618347, 1.0115917921066284, 0.37320467829704285, -0.38415729999542236, 0.011747356504201889, 0.14240433275699615,

## Clustering

### Apply K-Means clustering

Reference- [ML Clustering](https://spark.apache.org/docs/latest/ml-clustering.html).

##### Step 1: Apply `KMeans` Clustering

In [0]:
import time
# starting of Pre-Processing
start_time=time.time()

In [0]:
# Apply KMeans clustering using same seed value
kmeans = KMeans().setK(6).setSeed(1)\
        .setFeaturesCol("features")\
        .setPredictionCol("Cluster_no")

##### Step 2:  `fit` the model, and store in a `kmeansModel` variable.

In [0]:
#Fit the output dataframe and store it in the model variable.
kmeansModel1 = kmeans.fit(df2_train)
# Make predictions
df_clusters1=kmeansModel1.transform(df2_train)

### Number of majority and minority points in each cluster

Get the number of fraudulent and nonfraudulent transactions in each cluster

In [0]:
# define a function with input as 
# 1. df after kmeans clustering that contains cluster_no column (Dataframe)
# 2. max value in Cluster_no column (integer)
# returns dataframe with columns - Class , Cluster_no, count
def count_classes_in_cluster(df, total_clusters):
  for i in range(total_clusters):
    out = df.groupBy("label", "Cluster_no").count()
  return out

#count_classes_in_cluster(df_clusters1,5).sort(F.asc("Cluster_no")).collect() # 6 clusters

### Ratio of majority and minority points in each clusters

Define a function calculating the ratio of number of fraudulent transaction vs non fraudulent transaction i.e minority vs majority datapoints respectively. in each cluster

In [0]:
def cluster_ratio(df):
  
  '''
    gives a list of ratio of Class fraudulent and non- fraudulent in each cluster, given a spark df after clustering
    input:
    * df: df should contain columns 'Class' and  'Cluster_no'
    output:
    * list_of_ratio: list of values - with ratio if both classes are present, -1 if only class 1 is present and 0 if onlt class 0 is present
  '''
  
  list_of_ratio=[]
  for cluster in range(1,7):
    out = df.groupBy("label", "Cluster_no").count().filter(df.Cluster_no==cluster-1).sort(F.desc("label"))
    
    # Only one class is present in the cluster i.e. fraudulent or non fraudulent
    if out.count()!=2:
      
      # if non fraudulent
      if out.select("label").take(1)==0:
        list_of_ratio.append(0)
        
      # if fraudulent
      else:
        list_of_ratio.append(-1)
    
    # Both fraudulent and non fraudulent classes are present in the cluster
    else:
      out=out.select("count").take(2)
      list_of_ratio.append(out[0][0]/out[1][0])
      
  return list_of_ratio

cluster_ratio_list=cluster_ratio(df_clusters1)

In [0]:
df_clusters_1=df_clusters1.drop("Cluster_no")
df_cluster5 = df_clusters_1.filter("Cluster_no == 4")
df_cluster5.cache()
df_cluster4 = df_clusters_1.filter("Cluster_no == 3")
df_cluster4.cache()
df_cluster3 = df_clusters_1.filter("Cluster_no == 2")
df_cluster3.cache()
df_cluster2 = df_clusters_1.filter("Cluster_no == 1")
df_cluster2.cache()
df_cluster1 = df_clusters_1.filter("Cluster_no == 0")
df_cluster1.cache()
df_cluster6 = df_clusters_1.filter("Cluster_no == 5")
df_cluster6.cache()

Out[92]: DataFrame[features: vector, label: int]

In [0]:
df.writeTo("kk").partitionedBy(  

    bucket(K, "ts")

).createOrReplace()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2785966036098988> in <module> 
 1 df.writeTo("kk").partitionedBy( 
 2 
 ----> 3 bucket ( K , "ts" ) 
 4 
 5 ).createOrReplace()

 NameError : name 'bucket' is not defined

In [0]:
K=6

df_clusters1.write.format("parquet").bucketBy(K, "Cluster_no").saveAsTable("bucketed")

In [0]:
df_clusters1.writeTo("kk").partitionedBy(  

    bucket(K, "Cluster_no")

).createOrReplace()

In [0]:
t3 = spark.table("bucketed")
#t3.partitionedBy()
t3.rdd.getNumPartitions()
def f(t3):
  t3.map(smote)

t3.reduceByKey(smote)


## SMOTE + ENN with clusters 

- Each clusters will have first SMOTE applied and then ENN by checking each clusters ratio.

### Application of SMOTE on each cluster 
- SMOTE is applied only to clusters whose ratio is between 0 and 1

In [0]:
#configure SMOTE configuration
class conf:
  def __init__(self):
    self.seed = 1
    self.bucketLength = 1000
    self.k = 5
    self.multiplier = 100

In [0]:

conf_settings = conf()
# SMOTE applied to each cluster dataframe
if(0<cluster_ratio_list[0] and cluster_ratio_list[0]<1):
  oversampled_1=smote(df_cluster1,conf_settings).cache()
if(0<cluster_ratio_list[1] and cluster_ratio_list[1]<1):
  oversampled_2=smote(df_cluster2,conf_settings).cache()
if(0<cluster_ratio_list[2] and cluster_ratio_list[2]<1):
  oversampled_3=smote(df_cluster3,conf_settings).cache()
if(0<cluster_ratio_list[3] and cluster_ratio_list[3]<1):
  oversampled_4=smote(df_cluster4,conf_settings).cache()
if(0<cluster_ratio_list[5] and cluster_ratio_list[5]<1):
  oversampled_6=smote(df_cluster6,conf_settings).cache()
if(0<cluster_ratio_list[4] and cluster_ratio_list[4]<1):
  oversampled_5=smote(df_cluster5,conf_settings).cache()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 warnings.warn(
generating batch 0 of synthetic instances
generating batch 1 of synthetic instances
generating batch 2 of synthetic instances
generating batch 3 of synthetic instances
generating batch 4 of synthetic instances
generating batch 5 of synthetic instances
generating batch 6 of synthetic instances
generating batch 7 of synthetic instances
generating batch 8 of synthetic instances
generating batch 9 of synthetic instances
generating batch 10 of synthetic instances
generating batch 11 of synthetic instances
generating batch 12 of synthetic instances
generating batch 13 of synthetic instances
generating batch 14 of synthetic instances
generating batch 15 of synthetic instances
generating batch 16 of synthetic instances
generating batch 17 of synthetic instances
generating batch 18 of synthetic instances
generating batch 19 of synthetic instances
generating batch 20 of synthetic instances
generating batch 21 of synthetic instances
generating batch 22 of synthetic instances
generating batch 23 of synthetic instances
generating batch 24 of synthetic instances
generating batch 25 of synthetic instances
generating batch 26 of synthetic instances
generating batch 27 of synthetic instances
generating batch 28 of synthetic instances
generating batch 29 of synthetic instances
generating batch 30 of synthetic instances
generating batch 31 of synthetic instances
generating batch 32 of synthetic instances
generating batch 33 of synthetic instances
generating batch 34 of synthetic instances
generating batch 35 of synthetic instances
generating batch 36 of synthetic instances
generating batch 37 of synthetic instances
generating batch 38 of synthetic instances
generating batch 39 of synthetic instances
generating batch 40 of synthetic instances
generating batch 41 of synthetic instances
generating batch 42 of synthetic instances
generating batch 43 of synthetic instances
generating batch 44 of synthetic instances
generating batch 45 of synthetic instances
generating batch 46 of synthetic instances
generating batch 47 of synthetic instances
generating batch 48 of synthetic instances
generating batch 49 of synthetic instances
generating batch 50 of synthetic instances
generating batch 51 of synthetic instances
generating batch 52 of synthetic instances
generating batch 53 of synthetic instances
generating batch 54 of synthetic instances
generating batch 55 of synthetic instances
generating batch 56 of synthetic instances
generating batch 57 of synthetic instances
generating batch 58 of synthetic instances
generating batch 59 of synthetic instances
generating batch 60 of synthetic instances
generating batch 61 of synthetic instances
generating batch 62 of synthetic instances
generating batch 63 of synthetic instances
generating batch 64 of synthetic instances
generating batch 65 of synthetic instances
generating batch 66 of synthetic instances
generating batch 67 of synthetic instances
generating batch 68 of synthetic instances
generating batch 69 of synthetic instances
generating batch 70 of synthetic instances
generating batch 71 of synthetic instances
generating batch 72 of synthetic instances
generating batch 73 of synthetic instances
generating batch 74 of synthetic instances
generating batch 75 of synthetic instances
generating batch 76 of synthetic instances
generating batch 77 of synthetic instances
generating batch 78 of synthetic instances
generating batch 79 of synthetic instances
generating batch 80 of synthetic instances
generating batch 81 of synthetic instances
generating batch 82 of synthetic instances
generating batch 83 of synthetic instances
generating batch 84 of synthetic instances
generating batch 85 of synthetic instances
generating batch 86 of synthetic instances
generating batch 87 of synthetic instances
generating batch 88 of synthetic instances
generating batch 89 of synthet

In [0]:
df_smote_append = oversampled_1.union(oversampled_2).cache()\
                                   .union(oversampled_3).cache()\
                                   .union(oversampled_4).cache()\
                                   .union(oversampled_5).cache()\
                                   .union(oversampled_6).cache()

### Application of ENN to each clusters

- ENN is applied only to clusters whose ratio is between 0 and 1 or else assigned to original cluster dataframe

In [0]:
# configure ENN configuration
class conf1:
  def __init__(self):
    self.seed = 2
    self.bucketLength = 10
    self.k = 3

conf_settings = conf1()

In [0]:
conf_settings = conf1()
df_smote_enn_append=enn(df_smote_append,conf_settings).cache()

conf_settings = conf1()
# ENN applied to each cluster dataframe
if(0<cluster_ratio_list[0] and cluster_ratio_list[0]<1):
  df_enn_1=enn(oversampled_1,conf_settings).cache()
  #df_enn_1.show(10)
else:
  df_cluster1 = df_cluster1.select(
    "label",
    "features")
  df_enn_1=df_cluster1
if(0<cluster_ratio_list[1] and cluster_ratio_list[1]<1):
  df_enn_2=enn(oversampled_2,conf_settings).cache()
  #df_enn_2.show(10)
else:
  df_cluster2 = df_cluster2.select(
    "label",
    "features")
  df_enn_2=df_cluster2
if(0<cluster_ratio_list[2] and cluster_ratio_list[2]<1):
  df_enn_3=enn(oversampled_3,conf_settings).cache()
  #df_enn_3.show(10)
else:
  df_cluster3 = df_cluster3.select(
    "label",
    "features")
  df_enn_3=df_cluster3
if(0<cluster_ratio_list[3] and cluster_ratio_list[3]<1):
  df_enn_4=enn(oversampled_4,conf_settings).cache()
  #df_enn_4.show(10)
else:
  df_cluster4 = df_cluster4.select(
    "label",
    "features")
  df_enn_4=df_cluster4
if(0<cluster_ratio_list[4] and cluster_ratio_list[4]<1):
  df_enn_5=enn(oversampled_5,conf_settings).cache()
  #df_enn_5.show(10)
else:
  df_cluster5 = df_cluster5.select(
    "label",
    "features")
  df_enn_5=df_cluster5
if(0<cluster_ratio_list[5] and cluster_ratio_list[5]<1):
  df_enn_6=enn(oversampled_6,conf_settings).cache()
else:
  df_cluster6 = df_cluster6.select(
    "label",
    "features")
  df_enn_6=df_cluster6
# Combining all dataframe to one dataframe
df_smote_enn_append = df_enn_1.union(df_enn_2).cache()\
                                   .union(df_enn_3).cache()\
                                   .union(df_enn_4).cache()\
                                   .union(df_enn_5).cache()\
                                   .union(df_enn_6).cache()
#df_enn_6.show(10)

# Evaluation

- Evaluating our pre-processed data

In [0]:
# to calculate the end time of the whole pre-processing which includes clustering to SMOTE+ENN
df_smote_enn_append.show(10)
end_time=time.time()

+-----+--------------------+
label| features|
+-----+--------------------+
 0|[60599.0,0.256465...|
 0|[60600.0,-1.12148...|
 0|[60600.0,1.165764...|
 0|[60600.0,1.258347...|
 0|[60601.0,1.051520...|
 0|[60602.0,-0.87602...|
 0|[60607.0,-1.03985...|
 0|[60607.0,1.326237...|
 0|[60607.0,1.421944...|
 0|[60609.0,-2.50266...|
+-----+--------------------+
only showing top 10 rows

## Time taken for pre-processing

- Start time - End time

In [0]:
-start_time+end_time

Out[101]: 1883.0413084030151

## Ratio of pre-processed data

- Ratio  of minority and majority classes after pre-processing

In [0]:
# calculating the ratio  of minority samples to majority samples
df_smote_enn_append.filter(df_smote_enn_append.label==1).count()/df_smote_enn_append.filter(df_smote_enn_append.label==0).count()

Out[102]: 0.1440309681189059

## Evaluation of the Random Forest model using the pre-process data

- Creating a random forest model and fitting it with test data
- Running different evaluation metrics of the data

### Creating a  model and fitting it with test data

- Training a Random Forest Classifier model with pre-processed data and predicting the values with the test data

In [0]:
# training the pre-processed data with SMOTE+ENN using Random Forest
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
model = classifier.fit(df_smote_enn_append)
predictions = model.transform(df2_test)

### Evaluation the model

The metrics used:
- Area Under ROC
- Area Under Precision and Recall
- Accuracy of the model
- Precision of the model
- Recall of the model
- F1 score of the model

In [0]:
# Evaluating the model test data
binary_evaluator = BinaryClassificationEvaluator(labelCol ="label")

# Area Under ROC
print("Test Area Under ROC: " + str(binary_evaluator.evaluate(predictions, {binary_evaluator.metricName: "areaUnderROC"})))

# Area Under Precision and Recall
print("Test Area Under PR: " + str(binary_evaluator.evaluate(predictions, {binary_evaluator.metricName: "areaUnderPR"})))

#Accuracy of the model
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
print("Accuracy: " + str(evaluator_accuracy.evaluate(predictions)))

#Precision of the model
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="precisionByLabel")
print("Precision: " + str(evaluator_precision.evaluate(predictions)))

#Recall of the model
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="recallByLabel")
print("Recall: " + str(evaluator_recall.evaluate(predictions)))

#F1 score of the model
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
print("F1: " + str(evaluator_f1.evaluate(predictions)))



Test Area Under ROC: 0.952126091391781
Test Area Under PR: 0.7842314253952333
Accuracy: 0.9992607455908755
Precision: 0.9997648442092887
Recall: 0.9994945517379192
F1: 0.999287345855965